In [1]:
import fastai
from fastai.vision.all import (
    DataLoaders, Learner, URLs, cnn_learner, resnet18, 
    accuracy, ImageDataLoaders, untar_data
)
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F

from mnist.mean_classifier import MeanClassifier, mse, mae
from mnist import sgd, learner, loss
from mnist import data

In [2]:
train_dl, test_dl = data.load_mnist()
dataloaders = DataLoaders(train_dl, test_dl)

regression_dl = data.load_linear()

In [3]:
for loss_func in mse, mae:
    clf = MeanClassifier(loss=loss_func)
    clf.fit(train_dl)
    print(f'Mean Classifier with {loss_func.__name__.upper()} loss had an accuracy of {clf.score(test_dl):.4}')

Mean Classifier with MSE loss had an accuracy of 0.9661
Mean Classifier with MAE loss had an accuracy of 0.9514


# Gradient Descent

In [4]:
for loss_func in F.mse_loss, F.l1_loss:
    reg = sgd.SGDRegressor(loss=loss_func, epochs=100, lr=1e-3)
    reg.fit(regression_dl)
    print(f'Gradient Descent Regressor with {loss_func.__name__.upper()} loss had an R^2 of {reg.score(regression_dl):.4}')

Gradient Descent Regressor with MSE_LOSS loss had an R^2 of 0.9812
Gradient Descent Regressor with L1_LOSS loss had an R^2 of 0.9778


# Classifier

In [5]:
clf = sgd.SGDClassifier(lr=1, epochs=10)
clf.fit(train_dl)
print(f'Gradient Descent Classifier with custom SGD had an accuracy of {clf.score(test_dl):.4}')

Gradient Descent Classifier with custom SGD had an accuracy of 0.9764


In [6]:
# trying with pytorch's SGD:
clf = learner.Learner(epochs=10, lr=1, optimiser=torch.optim.SGD, loss=loss.sigmoid_loss, model=torch.nn.Linear(28*28, 1))
clf.fit(train_dl)
score = np.mean([sgd.accuracy(clf.predict(X), y) for X, y in test_dl])
print(f"Gradient Descent Classifier with torch's SGD had an accuracy of {score:.4}")

Gradient Descent Classifier with torch's SGD had an accuracy of 0.9814


In [7]:
learn = Learner(dataloaders, torch.nn.Linear(28*28,1), opt_func=fastai.optimizer.SGD, loss_func=loss.sigmoid_loss, metrics=sgd.accuracy)
learn.fit(10, lr=1)

epoch,train_loss,valid_loss,accuracy,time
0,0.057708,0.041235,0.970559,00:00
1,0.039786,0.034884,0.973994,00:00
2,0.032156,0.031623,0.976447,00:00
3,0.028182,0.029790,0.977429,00:00
4,0.025642,0.028267,0.978410,00:00
5,0.023651,0.028110,0.979392,00:00
6,0.022271,0.026612,0.979882,00:00
7,0.020976,0.025542,0.981354,00:00
8,0.020120,0.024972,0.981354,00:00
9,0.019817,0.024646,0.980373,00:00


# Moving to Neural Networks

In [8]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1),
    nn.Sigmoid()
)

In [9]:
clf = sgd.SGDClassifier(1, 10, loss=loss.sigmoid_loss, model=simple_net)
clf.fit(train_dl)
print(f"Neural Network had an accuracy of {clf.score(test_dl):.4}")

Neural Network had an accuracy of 0.975


In [ ]:
# and beyond
path = untar_data(URLs.MNIST_SAMPLE)

dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False,
                    loss_func=F.cross_entropy, metrics=accuracy)
learn.fit_one_cycle(1, 0.1)

epoch,train_loss,valid_loss,accuracy,time
